# **3.b Formación de valores A, B y C**

**Responsable:**

César Zamora Martínez

**Infraestructura usada:** 
Google Colab, para pruebas


## 0. Instalación de Cupy en Colab

In [4]:
!curl https://colab.chainer.org/install | sh -

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1580  100  1580    0     0   2573      0 --:--:-- --:--:-- --:--:--  2569
+ apt -y -q install cuda-libraries-dev-10-0
Reading package lists...
Building dependency tree...
Reading state information...
cuda-libraries-dev-10-0 is already the newest version (10.0.130-1).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.
+ pip install -q cupy-cuda100  chainer 
+ set +ex
Installation succeeded!


## 1. Implementación

**Consideraciones:**. Esta etapa supone que se conocen $\mu$ y $\Sigma$ asociados a los activos. El fin de este paso es obtener valores escalares que serán relevantes para obtener los pesos del portafolio para el inversionista.

Lo cual, se lleva a cabo en concreto a través de las expresiones:

$$A = \mu^t \cdot \Sigma^{-1} \cdot \mu $$

$$B = 1^t \cdot \Sigma^{-1} \cdot 1 $$

$$C = 1^t \cdot \Sigma^{-1} \cdot \mu $$


En este tenor a continución se presenta el código correspondiente:



In [0]:
import cupy as cp

def formar_abc(mu, Sigma):
  '''
  Calcula las cantidades A, B y C del diagrama de flujo del problema de Markowitz

  Args:
    mu (cupy array, vector): valores medios esperados de activos (dimension n)
    Sigma (cupy array, matriz): matriz de covarianzas asociada a activos (dimension n x n)

  Return:
    A (cupy array, escalar): escalar dado por mu^t \cdot Sigma^-1 \cdot mu
    B (cupy array, escalar): escalar dado por 1^t \cdot Sigma^-1 \cdot 1
    C (cupy array, escalar): escalar dado por 1^t \cdot Sigma^-1 \cdot mu
  '''

  # Vector auxiliar con entradas igual a 1
  n = Sigma.shape[0]
  ones_vector = cp.ones(n)

  # Formamos vector \cdot Sigma^-1 mu y Sigm^-1 1
  # Nota: 
  #   1) u= Sigma^-1 \cdot mu se obtiene resolviendo  Sigma u = mu
  #   2) v= Sigma^-1 \cdot 1 se obtiene resolviendo  Sigma v = 1

  u = cp.linalg.solve(Sigma, mu)
  v = cp.linalg.solve(Sigma, ones_vector)

  # Obtiene escalares de interes
  A = mu.transpose()@u
  B = ones_vector.transpose()@v
  C = ones_vector.transpose()@u

  return A, B, C

## 1.1 Matrices de prueba

In [0]:
n= 10**4

Sigma=cp.random.rand(n, n)
mu=cp.random.rand(n, 1)

In [7]:
formar_abc(mu, Sigma)

(array([[154.42787347]]), array(1.78220947), array([-12.51604798]))